# cadCAD model for Gift Economies of Scale


author: raluca diugan

This notebook contains the system model, the simulation setup, and initial analyses for the Gift Economies of Scale project. 

# Partial State Update Blocks
The figure below contains the main components of the system model in terms of state update functions per timestep, in order of execution.

In [ ]:
from PIL import Image
from IPython.display import display

img = Image.open('../artifacts/geos_psubs_v.0.2.png')
display(img)

In [ ]:
# imports
import sys
import csv

import random
import operator 

import numpy as np
import pandas as pd
import networkx as nx

import matplotlib.animation
import matplotlib.pyplot as plt 

from collections import defaultdict

# cadCAD-specific imports
from cadCAD import configs

from cadCAD.configuration import Experiment
from cadCAD.configuration.utils import config_sim

from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor


# import geos-specific classes
sys.path.append('../module')

from geos import *

In [ ]:
# configurations

# analysis
plt.rcParams['font.family'] = 'monospace'


In [ ]:
# state macros 

## request
REQUEST_EXPIRED = "expired"
REQUEST_SUBMITTED = "submitted"
REQUEST_FULFILLED = "fulfilled"
SUBREQUEST_FULFILLED = "fulfilled"

## donation 
DONATION_SELECTED = "selected"
DONATION_FINALIZED = "finalized"
DONATION_SUBMITTED = "submitted"

## solver
SOLVER_BREAKDOWN = "breakdown"
SOLVER_VALIDATION = "validation"
SOLVER_MATCHMAKING = "matchmaking"

## receipts
RECEIPT_DONOR = "donor"
RECEIPT_REQUESTOR = "requestor"

## agents & solvers
SOLVER_TYPES = ["breakdown", "matchmaking", "validation"]
AGENT_TYPES = ["decentralization-conscious", "honest", "rational"]

## substep 1

In [ ]:
# policy functions

def p_update_inventory_policy(params, substep, state_history, previous_state):
    '''
    updates agents' inventories across three dimensions:
    1. consumption of resources
    2. acquisition of resources
    3. TODO inventory policy (increase/decrease idle stock of resource)
    '''

    agents = previous_state['agents']
    inventory = previous_state['inventory']

    new_agent_stocks = defaultdict(lambda: [])
    new_inventory = defaultdict(lambda: {})
    consumption = defaultdict(lambda: [])

    # (1) consumption
    for agent_id, agent in agents.items():

        consumption_count = random.randint(0, 5)#(0, 5) # TODO system parameter
        consumption_choices = random.sample(list(agent.inventory.stock.keys()), consumption_count)

        for resource_id in consumption_choices:
            if agent.inventory.stock[resource_id]["quantity"] > 0: # consume if enough stock
                consumption_quantity = random.randint(1, agent.inventory.stock[resource_id]["quantity"]) # consume from quantity (i.e., in-use, not idle stock)
                consumption[agent_id].append({
                    "resource_id": resource_id,
                    "quantity": consumption_quantity
                })

    # (2) external acquisition 
    for agent_id, agent in agents.items():

        acquisition_count = random.randint(0, 5) # TODO system parameter
        acquisition_choices = random.sample(list(inventory.stock.keys()), acquisition_count)

        for resource_id in acquisition_choices:
            if inventory.stock[resource_id]["idle_stock"] > 1:

                # TODO update: right now qty cannot go to 0 (else division by zero in metrics) s.t. min qty of global stock is 1
                # acquisition only from idle stock
                acquisition_quantity = random.randint(1, min(inventory.stock[resource_id]["idle_stock"] - 1, 500)) 

                inventory.stock[resource_id]["idle_stock"] -= acquisition_quantity # local update, not applied to state
                inventory.stock[resource_id]["quantity"] -= acquisition_quantity # local update, not applied to state
            
                new_inventory[resource_id] = {
                    "quantity": inventory.stock[resource_id]["quantity"], 
                    "idle_stock": inventory.stock[resource_id]["idle_stock"]
                }
         
                new_agent_stocks[agent_id].append({
                    "resource_id": resource_id,
                    "quantity": acquisition_quantity
                })

    # TODO (3) usage strategy

    return {'new_agent_stocks': new_agent_stocks, 'new_inventory': new_inventory, 'consumption': consumption}

In [ ]:
# state update functions 

def s_update_inventory_policy(params, substep, state_history, previous_state, policy_input):
    '''
    applies updates to agent inventories
    '''

    resources = previous_state['inventory']
    agents_new = previous_state['agents'].copy()
    consumption = policy_input['consumption']
    new_agent_stocks = policy_input['new_agent_stocks']

    # apply consumption
    for agent_id, stocks in consumption.items():
        for stock in stocks:
            resource_id = stock["resource_id"]
            agents_new[agent_id].inventory.stock[resource_id]["quantity"] -= stock["quantity"]
   
    # apply acquisition
    for agent_id, stocks in new_agent_stocks.items():
        for stock in stocks:
            resource_id = stock["resource_id"]
            if resource_id in agents_new[agent_id].inventory.stock.keys():
                agents_new[agent_id].inventory.stock[resource_id]["quantity"] += stock["quantity"]
            else:
                agents_new[agent_id].inventory.stock[resource_id] = {
                    "resource": resources.stock[resource_id]["resource"],
                    "quantity": stock["quantity"],
                    "idle_stock": 0, 
                    "locked": 0
                }
            
    # TODO apply inventory policy changes
                
    return ('agents', agents_new)

def s_update_global_stocks(params, substep, state_history, previous_state, policy_input):
    '''
    updates global stocks following acquisition
    TODO add update based on natural resource decay
    '''

    inventory_new = previous_state['inventory'].make_copy()
    new_inventory = policy_input['new_inventory']

    for resource_id in new_inventory.keys():
        inventory_new.stock[resource_id]["quantity"] = new_inventory[resource_id]["quantity"]
        inventory_new.stock[resource_id]["idle_stock"] = new_inventory[resource_id]["idle_stock"]

    return ('inventory', inventory_new)

## substep 2

In [ ]:
# helper functions

def get_new_index(resource, resource_id, agents, qty, requestor, donor, len_agents):
    '''
    calculates the expected decentralization index of a given resource
    based on the donation decision of a potential (decentralization-conscious) donor
    '''

    if resource_id in agents[requestor].inventory.stock.keys():
        agents[requestor].inventory.stock[resource_id]["quantity"] += qty
    else:
        agents[requestor].inventory.stock[resource_id] = {
            "resource": resource,
            "quantity": qty, 
            "idle_stock": 0, 
            "locked": 0
        }
    agents[donor].inventory.stock[resource_id]["quantity"] -= qty
   
    ci, min_c, max_c = calculate_concentration_index(resource_id, agents)
    di = calculate_distribution_index(resource_id, agents, len_agents)
    index = calculate_decentralization_index(di, ci)

    return index

In [ ]:
# policy functions

def p_submit_request(params, substep, state_history, previous_state):
    '''
    submit request for resource
    current implementation assigns request to solver within subeconomy (or global)
    '''

    agents = previous_state['agents']
    requests = previous_state['requests'] 
    resources = previous_state['inventory'].stock 

    need_threshold = params['need_threshold']
    solvers_by_economy = previous_state['solvers_by_economy']
    
    complex_requests = {}

    for agent_id in agents:
        need = np.random.normal(size=1)[0] # determine need, i.e., if request will be submitted

        if need >= need_threshold:
            
            # prepare request data
            request_id = "request_" + str(len(requests))

            # randomize resource needed and quantity
            resource_id = random.choice(list(resources.keys()))
            quantity = np.random.randint(1, 20) # TODO system param (alternative: max resource qty at given time in system; in this case must handle concurrent requests)
            economy_id = random.choice(agents[agent_id].economies) # send to one of the economies in which the agent belongs
            solver_id = random.choice(solvers_by_economy[economy_id]) # pick solver for request
            deadline = len(state_history) + random.randint(5, 20) # TODO system param
            
            # compile request
            request = Request(request_id, resource_id, quantity, agent_id, [], resources[resource_id]['resource'].rtype, solver_id, deadline, economy_id)

            requests[request_id] = request

            # for complex requests, prepare for request breakdown in next substep
            if request.rtype == "complex":
                complex_requests[request_id] = request

    return {'new_requests': requests, 'pending_requests': complex_requests}


def p_donation_response(params, substep, state_history, previous_state):
    '''
    compile donation responses to requests in previous states
    each agent queries all requests and determines whether they can donate (acc. to their inventory policy)
    '''

    agents = previous_state['agents']
    requests = previous_state['requests']
    donations = previous_state['donation_responses']
    metrics = previous_state['metrics']

    donation_responses = {}

    for agent_id, agent in agents.items():

        for request_id, request in requests.items():
            if request.state == REQUEST_SUBMITTED: # only active requests, i.e., not fulfilled or expired
                # check that current agent is not already donor to the same request
                # TODO move to function
                already_donated = False 
                for donation_id, donation in donations.items(): # TODO optimize (e.g., keep list of donors for request)
                    if donation.request_id == request_id:
                        if donation.donor == agent_id:
                            already_donated = True

                if not already_donated:

                    # only atomic requests/subrequests (complex ones are solved through corresponding subrequests)
                    if request.rtype == "atomic" or request.rtype == "subrequest":
                        if agent_id != request.requestor: # exclude self-donations # TODO verify condition earlier
                            # TODO handle the donation of the same resource in the same timestep
                            if request.resource_id in agent.inventory.stock.keys(): # if agent holds the requested resource
                                # TODO modularize agent behavior based on type
                                # (a) decentralization conscious agents
                                if agent.atype == AGENT_TYPES[0]:
                                       
                                    # TODO optimize
                                    index_current = metrics['decentralization_index'][request.resource_id]["decentralization_index"]
                                    total_stock = agent.inventory.stock[request.resource_id]["quantity"] + agent.inventory.stock[request.resource_id]["idle_stock"]

                                    would_be_donated = min(request.quantity, total_stock)
                                    resource = agent.inventory.stock[request.resource_id]["resource"]

                                    # get expected index
                                    index_new = get_new_index(resource, request.resource_id, agents, would_be_donated, request.requestor, agent_id, len(agents))
                                    
                                    if index_new > index_current: # strictly greater (equal is analoguous to rational behavior)
                                        
                                        if total_stock > 0: # if agent has anything of the requested resource (both idle or in use)

                                            # prepare donation data
                                            donation_id = "donation_" + str(len(donations) + len(donation_responses))
                                            donation_quantity = min(request.quantity, total_stock) # donate from entire stock (except locked)
                                            
                                            if request.economy_id in agent.economies:
                                                economy_id_from = request.economy_id
                                                economy_id_to = request.economy_id
                                            else: # choose solver for the donation if the donor and requesting agents are not in the same subeconomy 
                                                economy_id_to = request.economy_id
                                                economy_id_from = agent.economies[0] # TODO randomize

                                            # compile donation
                                            donation = DonationResponse(donation_id, agent_id, request_id, donation_quantity, [], economy_id_from, economy_id_to)
                    
                                            donation_responses[donation_id] = donation

                                    # TODO otherwise behaves like standard rational agents

                                # (b) honest or rational
                                # TODO split, rational should account for cost of donation
                                elif agent.atype == AGENT_TYPES[1] or agent.atype == AGENT_TYPES[2]:
                                    # always donate when sufficient stock
                                    if agent.inventory.stock[request.resource_id]["idle_stock"] > 0: # if agent has anything of the requested resource
                                
                                        # prepare donation data
                                        donation_id = "donation_" + str(len(donations) + len(donation_responses))
                                        donation_quantity = min(request.quantity, agent.inventory.stock[request.resource_id]['idle_stock'])
                                            
                                        if request.economy_id in agent.economies:
                                            economy_id_from = request.economy_id
                                            economy_id_to = request.economy_id
                                        else:
                                            economy_id_to = request.economy_id
                                            economy_id_from = agent.economies[0] # TODO can also be random
                                    
                                        # compile donation
                                        donation = DonationResponse(donation_id, agent_id, request_id, donation_quantity, [], economy_id_from, economy_id_to)
                                        
                                        donation_responses[donation_id] = donation
    
    return {'pending_donations': donation_responses}


def p_donation_receipt(params, substep, state_history, previous_state):
    '''
    agents submit donation receipts (as requestor/donor)
    TODO: implement donation withdrawal/quantity alterations
    '''

    agents = previous_state['agents']
    requests = previous_state['requests']
    strategies = previous_state['strategies']
    receipts = previous_state['donation_receipts']
    
    pending_receipts = {}

    for agent_id in agents: 
        # each agent queries all strategies 
        # TODO optimize, e.g., strategy keeps list of agents
        for request_id, strategy_array in strategies.items(): 
            # for a given strategy, check whether agent has already sent a receipt
            receipt_sent = False
            for past_receipt_id in agents[agent_id].receipts:
                if receipts[past_receipt_id].request_id == request_id:
                    receipt_sent = True

            # if no receipt and the request submitted
            # TODO check if second condition still needed
            if receipt_sent == False and requests[request_id].state == REQUEST_SUBMITTED:
                # for the donations in the strategy for fulfilling the respective request
                for strategy in strategy_array:
                    # check whether the current agent is the donor or requestor (i.e., whether the donation has been made or request submitted by current agent)
                    if agent_id == strategy.donor or agent_id == requests[request_id].requestor:
        
                        # determine whether donor or requestor receipt (depends on who the agent is)
                        rtype = ""
                        if agent_id == strategy.donor: 
                            rtype = RECEIPT_DONOR
                        
                        # TODO: simplify logic
                        elif agent_id == requests[request_id].requestor: 
                            rtype = RECEIPT_REQUESTOR
                            
                        # prepare receipt data
                        quantity = strategy.quantity
                        receipt_id = "receipt_" + str(len(receipts) + len(pending_receipts))
                        solver_id = requests[request_id].solver_id
                        resource_id = requests[request_id].resource_id

                        # compile receipt; strategy_id is the corresponding donation id
                        receipt = DonationReceipt(receipt_id, agent_id, request_id, rtype, solver_id, quantity, resource_id, strategy.id)

                        pending_receipts[receipt_id] = receipt

    return {'pending_receipts': pending_receipts}


In [ ]:
# state update functions

def s_donation_response(params, substep, state_history, previous_state, policy_input):
    '''
    add new donation responses to state
    '''

    pending_donations = policy_input['pending_donations']
    donation_responses_new = previous_state['donation_responses'].copy()

    for donation_id, donation in pending_donations.items():
        donation_responses_new[donation_id] = donation 
    
    return ('donation_responses', donation_responses_new)


def s_donation_receipt(params, substep, state_history, previous_state, policy_input):
    '''
    add new receipts to state
    '''

    new_receipts = policy_input['pending_receipts']
    receipts_new = previous_state['donation_receipts'].copy()

    for receipt_id, receipt in new_receipts.items():
        receipts_new[receipt_id] = receipt

    return ('donation_receipts', receipts_new)

def s_donation_receipt_pool(params, substep, state_history, previous_state, policy_input):
    '''
    add new receipts to receipt pool
    '''

    new_receipts = policy_input['pending_receipts']
    receipts_new = previous_state['receipts_pool'].copy()
    
    for receipt_id, receipt in new_receipts.items():
        receipts_new[receipt_id] = receipt

    return ('receipts_pool', receipts_new)

def s_agent_receipts(params, substep, state_history, previous_state, policy_input):
    '''
    add new receipts to the respective agent's state
    '''
    
    new_receipts = policy_input['pending_receipts']
    agents_new = previous_state['agents'].copy()

    for receipt_id, receipt in new_receipts.items():
        agent_id = receipt.agent_id

        agents_new[agent_id].receipts.append(receipt_id)

    return ('agents', agents_new)

def s_new_request(params, substep, state_history, previous_state, policy_input):
    '''
    add new requests to state
    '''

    pending_requests = policy_input['new_requests']
    requests_new = previous_state['requests'].copy()

    for request_id in pending_requests.keys():
        requests_new[request_id] = pending_requests[request_id]

    return ('requests', requests_new)

def s_request_pool(params, substep, state_history, previous_state, policy_input):
    '''
    add new requests to the request pool (for request breakdown where applicable)
    '''

    pending_requests = policy_input['pending_requests']
    request_pool_new = previous_state['requests_pool'].copy()

    for request_id in pending_requests.keys():
        request_pool_new[request_id] = pending_requests[request_id]

    return ('requests_pool', request_pool_new)

## substep 3

In [ ]:
# helper functions

def build_strategy(options, required_quantity):
    '''
    build donation strategy given responses and the required quantity
    TODO add solving algorithms (1), handle constraints (2), handle overdonation in multi-donation strategies (3)
    '''

    initial_quantity = required_quantity

    strategy = []
    running_sum = 0

    # source: https://ioflood.com/blog/python-sort-dictionary-by-value/
    # sort donations by qty (decreasing order (optimizes for least amount of donations))
    quantities = dict(sorted(options.items(), key=operator.itemgetter(1), reverse=True))
   
    # keep selecting donations until required quantity is reached
    for donation_id, quantity in quantities.items():
       
        if running_sum >= initial_quantity:
            break
    
        strategy.append(donation_id)
        running_sum += quantity

    return strategy, initial_quantity - running_sum # also return the overdonated quantity


def build_strategy_by_economy(options, request):
    '''
    build strategy given donation responses
    optimize for quantity within the same subeconomy, quantity from external donations
    TODO add delay for building strategy
    '''

    economy_requestor = request.economy_id
    requested_quantity = request.quantity

    aggregated_quantity = 0
    candidate_donations = {}
    other_donations = {}
    s, s2 = [], []
    over, over2 = 0, 0

    for donation_id, donation in options.items():
        if donation.economy_id_from == economy_requestor: # if the donation comes from within the same economy
            aggregated_quantity += donation.quantity
            candidate_donations[donation_id] = donation.quantity
        else:
            other_donations[donation_id] = donation.quantity

    if aggregated_quantity >= requested_quantity: # if enough quantity for a donation within subeconomy, select candidates by descending quantity
        
        s, over = build_strategy(candidate_donations, request.quantity)
        
        return s, over
        
    else:   # if not sufficient donations within economy
        remaining_quantity = requested_quantity - aggregated_quantity 
        s2, over2 = build_strategy(other_donations, remaining_quantity - over) # over has been covered already with the overdonation

    for donation_id in s2:
        s.append(donation_id)

    return s, over2

In [ ]:
# policy functions

def p_request_breakdown(params, substep, state_history, previous_state):
    '''
    breaks down complex requests into subrequests according to the dependent resources (currently subrequrest qty is random and capped at main request qty)
    TODO increase resource dependency depth
    NOTE treatment of solvers not differentiated (all perform the same task in the same substep)
    '''

    inventory = previous_state['inventory']
    requests_pool = previous_state['requests_pool']
    
    cleared = []
    subrequests = {}
    
    for request_id, request in requests_pool.items():
       
        resource_id = request.resource_id
        ctr = 0 # determines subrequest id

        # for each dependency of the requested resource compile a new request
        for dependency in inventory.stock[resource_id]["resource"].dependencies:

            # prepare subrequest data
            resource_id = dependency
            subrequest_id = request_id + "_" + str(ctr)

            # TODO non-random subrequest qty
                # this would require request breakdown on the client side
            subrequest_quantity = random.choice(range(1, request.quantity + 1))
            
            # compile subrequest
            subrequest = Request(subrequest_id, resource_id, subrequest_quantity, request.requestor, [], "subrequest", request.solver_id, request.deadline, request.economy_id)
            
            # prepare to update the request's dependencies
            request.subrequests.append(subrequest)

            # prepare to update requests
            subrequests[subrequest_id] = subrequest

            ctr += 1

        # keep track of processed main requests
        cleared.append(request_id)

    return {'subrequests': subrequests, 'cleared': cleared}

def p_receipts_match(params, substep, state_history, previous_state):
    '''
    match receipts in receipts pool corresponding to the same request
    when a complete match is available (i.e., for all subrequests/donation pairs), mark request for fulfillment
    '''
    
    requests = previous_state['requests']
    strategies = previous_state['strategies']
    receipts = previous_state['receipts_pool']
    
    requests_receipts = {}
    fulfilled_requests = []
    donations_finalized = []

    # get requestor receipts
    for receipt_id, receipt in receipts.items():
        if receipt.rtype == RECEIPT_REQUESTOR:

            request_id = receipt.request_id

            requests_receipts[request_id] = {
                "expected_receipts": len(strategies[request_id]), 
                "expected_quantity": requests[request_id].quantity
            }

    # for each request with a requestor receipt look for matching donor receipt(s) (number and quantity)
    # TODO optimize
    for request_id in requests_receipts.keys():
    
        qty = 0
        donors = []
        donation_receipts_count = 0
        pending_donations = []
        
        for receipt_id, receipt in receipts.items():
            if request_id == receipt.request_id: # if matching receipt for request
                # if receipt from donor
                # if new donor than previous matching donor receipts
                if receipt.rtype == RECEIPT_DONOR and receipt.agent_id not in donors:
        
                    donors.append(receipt.agent_id)
                    donation_receipts_count += 1
                    qty += receipt.quantity 

                    pending_donations.append(receipt.donation_id)

        # if conditions for matching receipt(s) met 
        # TODO FIX qty should be equal after fix (not >=) (current implementation allowes overdonation by last donation in multi-donation strategies)
        if donation_receipts_count == requests_receipts[request_id]["expected_receipts"] and qty >= requests_receipts[request_id]["expected_quantity"]:
          
            fulfilled_requests.append(request_id) # keep track of fulfilled requests
            
            for donation_id in pending_donations:
                if donation_id not in donations_finalized:
                    donations_finalized.append(donation_id)

    return {'pending_requests_fulfilled': set(fulfilled_requests), 'finalized_donations': donations_finalized}

def p_donation_strategies(params, substep, state_history, previous_state):
    '''
    compile donation strategies for requests according to corresponding donation responses
    if individual donations do not suffice, compiles multi-donation strategies
    TODO add multiple strategy construction methods, integrate constraints in strategy preference (e.g., urgent requests)
    TODO optimize logic
    '''

    requests = previous_state['requests']
    pending_donations = previous_state['donation_responses']

    strategies = defaultdict(lambda: [])
    selected_strategies = defaultdict(lambda: [])

    # if no strategy for a given request
    # collect all corresponding donations
    for donation_id in pending_donations.keys():
        req_id = pending_donations[donation_id].request_id

        if requests[req_id].strategy_added == False: # if no existing strategy
            strategies[req_id].append(donation_id)

    # for each request for which a strategy does not exist
    for request_id in strategies.keys():
    
        # check if a strategy can be assembled, i.e., the sum of existing donation quantities is sufficient
        options = {}
        options_by_economy = {}
        sum_donations = 0

        # sum up all donation quantities
        for donation_id in strategies[request_id]:
            if pending_donations[donation_id].state != DONATION_SELECTED:

                options[donation_id] = pending_donations[donation_id].quantity
                options_by_economy[donation_id] = pending_donations[donation_id]
                sum_donations += pending_donations[donation_id].quantity
        
        if sum_donations >= requests[request_id].quantity:
    
            # prioritize by economy
            # TODO handle the excess donation (over_by_economy)
            solution_by_economy, over_by_economy = build_strategy_by_economy(options_by_economy, requests[request_id])
                        
            for donation_id in solution_by_economy: #s:
                selected_strategies[request_id].append(pending_donations[donation_id])
    
    return {'pending_strategies': selected_strategies}

In [ ]:
# state update functions

def s_donation_strategies(params, substep, state_history, previous_state, policy_input):
    '''
    apply new donation strategies if no strategy already exists
    TODO optimize (in policy) to only construct/apply new strategies
    '''

    strategies_new = previous_state['strategies'].copy()
    new_strategies = policy_input['pending_strategies']
    
    for request_id, donations in new_strategies.items():
        if request_id not in strategies_new.keys():
            strategies_new[request_id] = donations 
        
    return ('strategies', strategies_new)

def s_donation_responses_old(params, substep, state_history, previous_state, policy_input):
    '''
    update donation reponse state when selected for strategies
    '''

    strategies = policy_input['pending_strategies']
    donation_responses_new = previous_state['donation_responses'].copy()
    
    # mark donations selected in strategies
    for request_id, donations in strategies.items():
        for donation in donations:
            donation_responses_new[donation.id].state = DONATION_SELECTED
    
    return ('donation_responses', donation_responses_new)


def s_lock_quantities(params, substep, state_history, previous_state, policy_input):
    '''
    update agents' inventories to reflect quantities locked in strategies
    TODO simplify logic
    '''

    donors = previous_state['agents']
    requests = previous_state['requests']
    agents_new = previous_state['agents'].copy()
    strategies = policy_input['pending_strategies']

    for request_id, donations in strategies.items():
        for donation in donations: 
            request_id = donation.request_id
            resource_id = requests[request_id].resource_id

            # separately tackle stock updates for decentralization conscious agents
            if donors[donation.donor].atype == AGENT_TYPES[0]:
                # lock the full amount
                agents_new[donation.donor].inventory.stock[resource_id]["locked"] += donation.quantity # update locked stock

                # decrease idle_stock
                decrease_idle = min(donation.quantity, donors[donation.donor].inventory.stock[resource_id]["idle_stock"])
                agents_new[donation.donor].inventory.stock[resource_id]["idle_stock"] -= decrease_idle

                optional_remainder = donation.quantity - decrease_idle
                # also remove from in use stock
                if optional_remainder > 0:
                    agents_new[donation.donor].inventory.stock[resource_id]["quantity"] -= optional_remainder

            else:
                agents_new[donation.donor].inventory.stock[resource_id]["locked"] += donation.quantity # update locked stock
                agents_new[donation.donor].inventory.stock[resource_id]["idle_stock"] -= donation.quantity # decrease amount of available stock

    return ('agents', agents_new)

# TODO rename to reflect dual functionality
def s_request_breakdown(params, substep, state_history, previous_state, policy_input):
    '''
    updates requests (1) adds new subrequests (2) marks fulfilled requests (3) marks requests for which strategies exist
    TODO update expired requests (must implement request deadlines)
    '''
    
    subrequests = policy_input['subrequests']
    requests_new = previous_state['requests'].copy()
    strategies_added = policy_input['pending_strategies']
    pending_requests_fulfilled = policy_input['pending_requests_fulfilled']

    # add new requests (subrequests)
    for request_id in subrequests.keys():
        requests_new[request_id] = subrequests[request_id]

    # updated fulfilled requests
    for request_id in pending_requests_fulfilled:
        requests_new[request_id].state = REQUEST_FULFILLED

    # update requests with strategies
    for request_id in strategies_added.keys():
        requests_new[request_id].strategy_added = True

    return ('requests', requests_new)

def s_requests_fulfilled_pool(params, substep, state_history, previous_state, policy_input):
    '''
    updates the fulfilled requests pool to support stock updates in the next substep
    TODO simplify logic
    '''

    requests = previous_state['requests']
    requests_pool_new = previous_state['requests_fulfilled_pool'].copy()
    pending_requests_fulfilled = policy_input['pending_requests_fulfilled']

    for request_id in pending_requests_fulfilled:
        requests_pool_new[request_id] = requests[request_id]

    return ('requests_fulfilled_pool', requests_pool_new)

def s_receipts_pool_clear(params, substep, state_history, previous_state, policy_input):
    '''
    remove matched receipts
    TODO integrate present logic in receipts matching
    '''

    receipts_pool_new = previous_state['receipts_pool'].copy()
    pending_requests_fulfilled = policy_input['pending_requests_fulfilled']

    removable_receipts = []

    # find all obsolete receipts (that have been employed to fulfill a request)
    for request_id in pending_requests_fulfilled:
        for receipt_id, receipt in receipts_pool_new.items():
            if receipt.request_id == request_id:
                removable_receipts.append(receipt_id)
                
    # remove receipts from pool
    for receipt_id in removable_receipts:    
        del receipts_pool_new[receipt_id]

    return ('receipts_pool', receipts_pool_new)

def s_clear_pool(params, substep, state_history, previous_state, policy_input):
    '''
    clear complex requests pool
    '''

    cleared_requests =  policy_input['cleared']
    requests_pool_new = previous_state['requests_pool'].copy()
    
    for request_id in cleared_requests:
        del requests_pool_new[request_id]

    return ('requests_pool', requests_pool_new)

def s_finalized_donations_pool(params, substep, state_history, previous_state, policy_input):
    '''
    updated the pool of completed donations to support update stock logic
    TODO simplify process
    '''

    finalized_donations = policy_input['finalized_donations']
    finalized_donations_pool_new = previous_state['finalized_donations_pool'].copy()

    for donation_id in finalized_donations:
        finalized_donations_pool_new.append(donation_id)

    return ('finalized_donations_pool', finalized_donations_pool_new)


def s_finalize_donations(params, substep, state_history, previous_state, policy_input):
    ''' 
    update state of finalized donation responses
    '''

    donation_responses_new = previous_state['donation_responses'].copy()
    finalized_donations = policy_input['finalized_donations']

    for donation_id in finalized_donations:
        donation_responses_new[donation_id].state = DONATION_FINALIZED

    return ('donation_responses', donation_responses_new)

## substep 4

In [ ]:
# policy functions

def p_update_stocks(params, substep, state_history, previous_state):
    '''
    updates agents' inventories according to fulfilled requests (inc. if requestor, dec. if donor)
    TODO integrate 'donations' from system stock
    '''

    receipts = previous_state['donation_receipts']
    fulfilled_requests = previous_state['requests_fulfilled_pool']

    new_agents_stocks = defaultdict(lambda: [])
    
    # handles situations where an agent both donates and receives the same resource within the same timestep
    # for each fulfilled request, mark decrease (donor) or increase (requestor)
    # determine agent based on receipt rtype
    for request_id in fulfilled_requests:
        for receipt_id, receipt in receipts.items():
            if receipt.request_id == request_id:

                if receipt.rtype == RECEIPT_DONOR:
                    new_agents_stocks[receipt.agent_id].append({"resource_id": receipt.resource_id, "quantity": receipt.quantity, "operation": "decrease"})
                    
                elif receipt.rtype == RECEIPT_REQUESTOR:
                    new_agents_stocks[receipt.agent_id].append({"resource_id": receipt.resource_id, "quantity": receipt.quantity, "operation": "increase"})

    return {'new_agents_stocks': new_agents_stocks} 

def p_request_expired(params, substep, state_history, previous_state):
    '''
    identify expired requests based on state id
    '''

    st = len(state_history)
    requests = previous_state['requests']
    expired_requests = []

    for request_id, request in requests.items():
        if st > request.deadline and request.state == REQUEST_SUBMITTED: # exclude fulfilled requests
            
            expired_requests.append(request_id)

    return {'expired_requests': expired_requests}

In [ ]:
# state update functions 

def s_update_stocks(params, substep, state_history, previous_state, policy_input):
    '''
    apply changes to agents' stocks
    '''

    resources = previous_state['inventory']
    agents_new = previous_state['agents'].copy()
    new_agents_stocks = policy_input['new_agents_stocks']
    
    # for all changes to agents' stocks, apply as decrease/increase
    for agent_id, new_stocks in new_agents_stocks.items():
        # TODO verify correctness of decentralization conscious stock updates
        for new_stock in new_stocks:

            if new_stock["operation"] == "decrease": # decrease qty, idle_stock, and remove lock for donated qty
                agents_new[agent_id].inventory.stock[new_stock["resource_id"]]["quantity"] -= new_stock["quantity"]
                agents_new[agent_id].inventory.stock[new_stock["resource_id"]]["idle_stock"] -= new_stock["quantity"]
                agents_new[agent_id].inventory.stock[new_stock["resource_id"]]["locked"] -= new_stock["quantity"]

            elif new_stock["operation"] == "increase": # increase qty or add as new resource

                if new_stock["resource_id"] not in agents_new[agent_id].inventory.stock.keys(): # if new resource

                    resource_id = new_stock["resource_id"]

                    agents_new[agent_id].inventory.stock[new_stock["resource_id"]] = {
                        "resource": resources.stock[resource_id]["resource"],
                        "quantity": new_stock["quantity"], # received qty
                        "idle_stock": 0,
                        "locked": 0 # no donation pending
                        }
                    
                else: # if resource already in inventory, update qty; no change to idle stock/locked qty
                    agents_new[agent_id].inventory.stock[new_stock["resource_id"]]["quantity"] += new_stock["quantity"]

    return ('agents', agents_new)

def s_clear_requests_fulfilled_pool(params, substep, state_history, previous_state, policy_input):
    '''
    clear pool of fulfilled requests
    '''

    requests_fulfilled = previous_state['requests_fulfilled_pool']
    requests_fulfilled_pool_new = previous_state['requests_fulfilled_pool'].copy()
    
    for request_id in requests_fulfilled:
        del requests_fulfilled_pool_new[request_id]

    return ('requests_fulfilled_pool', requests_fulfilled_pool_new)


def s_request_expired(params, substep, state_history, previous_state, policy_input):
    '''
    update state of expired requests
    '''

    requests_expired = policy_input['expired_requests']
    requests_new = previous_state['requests'].copy()

    for request_id in requests_expired:
        requests_new[request_id].state = REQUEST_EXPIRED

    return ('requests', requests_new)

def s_clear_finalized_donations_pool(params, substep, state_history, previous_state, policy_input):
    ''' 
    NOTE pools cannot be cleared in the same substep
    TODO simplify
    '''

    return ('finalized_donations_pool', [])


## substep 5

In [ ]:
# helper functions
# TODO move to library

def calculate_average_request_fulfillment_latency(state_history):
    '''
    compute average latency of fulfilling requests
    TODO simplify
    '''
  
    latencies = []
    latency_by_request_fulfilled = {}

    for state_id in range(len(state_history)): # for each state
        for substep_id in range(len(state_history[state_id])):
            for request_id, request in state_history[state_id][substep_id]['requests'].items(): # for each request

                if request_id not in latency_by_request_fulfilled: # (for each request not accounted for yet)

                    latency_by_request_fulfilled[request_id] = {
                        "submitted": float(state_id), # state when submitted
                        "fulfilled": -1,
                    }
                
                # mark when request fulfilled
                if request.state == REQUEST_FULFILLED and latency_by_request_fulfilled[request_id]["fulfilled"] == -1:
                    latency_by_request_fulfilled[request_id]["fulfilled"] = float(state_id) 

    # find latency per request
    for request_id in latency_by_request_fulfilled.keys():
        if latency_by_request_fulfilled[request_id]["fulfilled"] > -1:

            latency = latency_by_request_fulfilled[request_id]["fulfilled"] - latency_by_request_fulfilled[request_id]["submitted"]
            latencies.append(latency)

    # if no requests fulfilled (for initial state)
    if len(latencies) == 0:
        return 0.0

    return round(sum(latencies) / len(latencies), 2) # avg latency

In [ ]:
# policy update functions

def p_update_metrics(params, substep, state_history, previous_state):
    '''
    compute latest metrics
    '''

    agents = previous_state['agents']
    requests = previous_state['requests']
    resources = previous_state['inventory']
    
    pending_metrics = {}
    requests_fulfilled = 0
    decentralization_indices = defaultdict(lambda: {})

    # requests fulfilled
    for request_id, request in requests.items():
        if request.state == REQUEST_FULFILLED:
            requests_fulfilled += 1
    
    # set new throughput
    pending_metrics['throughput'] = requests_fulfilled
   
    # compute new idling capacities
    idling_capacity_overall, idling_capacity_by_resource = calculate_cumulative_idling_capacity(previous_state['inventory'], previous_state['agents'])

    # set new idling capacities
    pending_metrics['cumulative_idling_capacity'] = idling_capacity_overall
    pending_metrics['cumulative_idling_capacity_by_resource'] = idling_capacity_by_resource

    # compute and set latency
    pending_metrics['latency'] = calculate_average_request_fulfillment_latency(state_history)
  
    for resource_id in inventory.stock.keys():

        decentralization_indices[resource_id] = {}
        ci, min_c, max_c = calculate_concentration_index(resource_id, agents)
        di = calculate_distribution_index(resource_id, agents, len(agents))
        decentralization_indices[resource_id]["concentration_index"] = ci
        decentralization_indices[resource_id]["distribution_index"] = di
        decentralization_indices[resource_id]["decentralization_index"]= calculate_decentralization_index(di, ci)
        decentralization_indices[resource_id]["min_c"] = round(min_c, 2)
        decentralization_indices[resource_id]["max_c"] = round(max_c, 2)

    # set decentralization indices
    pending_metrics['decentralization_index'] = decentralization_indices

    return {'pending_metrics': pending_metrics}

In [ ]:
# state update functions 

def s_update_metrics(params, substep, state_history, previous_state, policy_input):
    '''
    apply updates to metrics
    '''

    metrics_new = previous_state['metrics'].copy()
    pending_metrics = policy_input['pending_metrics']
   
    for metric_id in pending_metrics.keys():
        metrics_new[metric_id] = pending_metrics[metric_id]

    return ('metrics', metrics_new)

## Simulation config

In [ ]:

PSUBs = [
    {
        "policies": {
            'p_update_inventory_policy': p_update_inventory_policy,
        },
        "variables": {
            'agents': s_update_inventory_policy, 
            'inventory': s_update_global_stocks,
        }
    }, # substep 1
    {
        "policies": {
            'p_submit_request': p_submit_request,
            'p_donation_response': p_donation_response,
            'p_donation_receipt': p_donation_receipt,
            
        },
        "variables": {
            'requests': s_new_request,
            'requests_pool': s_request_pool,
            'donation_responses': s_donation_response,
            'donation_receipts': s_donation_receipt, 
            'receipts_pool': s_donation_receipt_pool,
            'agents': s_agent_receipts,
        }
    }, # substep 2
    {
        "policies": {
            'p_request_breakdown': p_request_breakdown,
            'p_donation_strategies': p_donation_strategies,
            'p_receipts_match': p_receipts_match,
        },
        "variables": {
            'requests': s_request_breakdown,
            'requests_pool': s_clear_pool,
            'strategies': s_donation_strategies,
            'donation_responses': s_donation_responses_old,
            'receipts_pool': s_receipts_pool_clear,
            'requests_fulfilled_pool': s_requests_fulfilled_pool,
            'agents': s_lock_quantities,
            'finalized_donations_pool': s_finalized_donations_pool,
            'donation_responses': s_finalize_donations,
        }
    }, # substep 3
    {
        "policies": {
            'p_update_stocks': p_update_stocks,
            'p_request_expired': p_request_expired,
        },
        "variables": {
            'requests': s_request_expired,
            'agents': s_update_stocks,
            'requests_fulfilled_pool': s_clear_requests_fulfilled_pool,
            'finalized_donations_pool': s_clear_finalized_donations_pool,
        }
    }, # substep 4
    {
        "policies": {
            'p_update_metrics': p_update_metrics,
        },
        "variables": {
            'metrics': s_update_metrics
        }
    }, # substep 5
]

In [ ]:
# initialization data

MAX_QTY_RESOURCES = 20000
COUNT_RESOURCES = 100

initial_state_params = {
    'need_threshold': [0.8],
    'count_economies': 5,
    'count_agents': 20,
    'count_solvers': 5,
    'count_resources': COUNT_RESOURCES, 
    'count_resource_dependencies': 3,
    'max_qty_resources': MAX_QTY_RESOURCES,
    'min_qty_resources': 50,
    'max_resource_count_agent': 20,
    'min_resource_count_agent': 5,
    'max_qty_resources_agent': 10,
    'count_resources_1': int(0.3 * COUNT_RESOURCES),
    'count_resources_2': int(0.7 * COUNT_RESOURCES),
    'agent_types_probabilities': [0.5, 0.4, 0.1], # decentralization-conscious, honest, rational
    'solver_types_probabilities': [0.3, 0.7], # whether the solver is global
    'resource_dependency_probabilities': [0.7, 0.3] #
}


In [ ]:
# initialize variables

economies = ["econ_" + str(i) for i in range(initial_state_params['count_economies'])]

agents = init_agents(economies, initial_state_params['count_agents'], initial_state_params['agent_types_probabilities'])
solvers = init_solvers(economies, initial_state_params['count_solvers'], initial_state_params['solver_types_probabilities'])

inventory = Inventory()
resources = init_resources(initial_state_params['count_resources_1'], initial_state_params['count_resources_2'], initial_state_params['resource_dependency_probabilities'])

for resource_id, resource in resources.items():
    quantity = random.randint(initial_state_params['min_qty_resources'], initial_state_params['max_qty_resources'])
    inventory.add_resource(resource_id, resource, quantity, quantity) # full stock availability


agents, inventory = distribute_inventory(agents, initial_state_params['min_resource_count_agent'], initial_state_params['max_resource_count_agent'], inventory)
idling_capacity_overall, idling_capacity_by_resource = calculate_cumulative_idling_capacity(inventory, agents)

In [ ]:
# TODO wrap as function (also apply to metrics update policy)
                                          
decentralization_indices = defaultdict(lambda: {})

for resource_id in inventory.stock.keys():
    decentralization_indices[resource_id] = {}
    (ci, min_c, max_c) = calculate_concentration_index(resource_id, agents)
    di = calculate_distribution_index(resource_id, agents, initial_state_params['count_agents'])
    decentralization_indices[resource_id]["concentration_index"] = ci
    decentralization_indices[resource_id]["distribution_index"] = di
    decentralization_indices[resource_id]["decentralization_index"]= calculate_decentralization_index(di, ci)
    decentralization_indices[resource_id]["min_c"] = round(min_c, 2)
    decentralization_indices[resource_id]["max_c"] = round(max_c, 2)

In [ ]:
# TODO: wrap as function

solvers_by_economy = defaultdict(lambda: [])

for economy in economies:
    for solver_id, solver in solvers.items():
        if economy in solver.economies:
            solvers_by_economy[economy].append(solver_id)
        if 'global' in solver.economies:
            if solver_id not in solvers_by_economy['global']:
                solvers_by_economy['global'].append(solver_id)
    solvers_by_economy[economy] = list(set(solvers_by_economy[economy]))

In [ ]:
# initial state

initial_state = {
    'requests': {}, # resource requests by their id
    'donation_responses': {}, # all announced donations by id
    'inventory': inventory, # available (undistributed) resources
    'agents': agents, # agents with their inventories; includes solvers
    'donation_receipts': {}, # proofs of donation by request id
    'strategies': {}, # selected donation(s) by request id
    'solvers': solvers,
    'solvers_by_economy': solvers_by_economy,
    'requests_pool': {},
    'receipts_pool': {},
    'requests_fulfilled_pool': {},
    'finalized_donations_pool': [],
    'metrics': {
        'throughput': 0, # total requests solved
        'average_latency': 0.0, # average request resolution over timesteps
        'decentralization_index': decentralization_indices, # concentration, distribution, and decentralization indices by resource
        'cumulative_idling_capacity': idling_capacity_overall, # idle / available (the lower the better as resources are used; however, open requests must be considered for discussion)
        'cumulative_idling_capacity_by_resource': idling_capacity_by_resource,
        'poa': None, # TBD
        'waste_rate': None, # TBD
        'waste_units': None, # TBD
    }
}

print(f"Initial State: {initial_state}")

# Simulation

In [ ]:
TIMESTEPS = 20

# TODO add system setup params
system_params = {
    'need_threshold': [0.8]
}

sim_config = config_sim({
    "N": 1, # Monte Carlo runs
    "T": range(TIMESTEPS), # timesteps
    "M": system_params # parameters
})

In [ ]:
del configs[:]

In [ ]:
experiment = Experiment()

experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = PSUBs,
    sim_configs = sim_config
)

In [ ]:
exec_context = ExecutionContext()
simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()

In [ ]:
# source: cadCAD notebooks

df = pd.DataFrame(raw_result)

# Insert cadCAD parameters for each configuration into DataFrame
for config in configs:
    # Get parameters from configuration
    parameters = config.sim_config['M']
    # Get subset index from configuration
    subset_index = config.subset_id
    
    # For each parameter key value pair
    for (key, value) in parameters.items():
        # Select all DataFrame indices where subset == subset_index
        dataframe_indices = df.eval(f'subset == {subset_index}')
        # Assign each parameter key value pair to the DataFrame for the corresponding subset
        df.loc[dataframe_indices, key] = value

df.head(5)

# Analysis

In [ ]:
# helper functions
# TODO refactor and move to module

def get_requests_stats(run_start, run_end):

    total_requests = []
    total_fulfilled_requests = []
    requests_stats_prc = {}

    for i in range(run_start, run_end + 1, 5):
        
        total_requests.append(len(df['requests'][i]))

        count_fulfilled = 0
        for request_id, request in df['requests'][i].items():
            if request.state == REQUEST_FULFILLED:
                count_fulfilled += 1

        total_fulfilled_requests.append(count_fulfilled)

    requests_stats = defaultdict(lambda: [])

    for i in range(run_start, run_end + 1, 5):
        requests_stats["total_requests"].append(len(df['requests'][i]))

        count_fulfilled = 0
        count_main = 0
        count_subrequests = 0
        count_main_fulfilled = 0
        count_subrequests_fulfilled = 0

        for request_id, request in df['requests'][i].items():
            if request.rtype == "atomic" or request.rtype == "complex":
                count_main += 1 

                if request.state == REQUEST_FULFILLED:
                    count_main_fulfilled += 1
                    count_fulfilled += 1

            else:
                count_subrequests += 1

                if request.state == REQUEST_FULFILLED:
                    count_subrequests_fulfilled += 1
                    count_fulfilled += 1

        requests_stats["fulfilled_requests"].append(count_fulfilled)
        requests_stats["fulfilled_main_requests"].append(count_main_fulfilled)
        requests_stats["fulfilled_subrequests"].append(count_subrequests_fulfilled)
        requests_stats["total_main_requests"].append(count_main)
        requests_stats["total_subrequests"].append(count_subrequests)

    requests_stats_prc["total_requests_prc"] = 100.0
    requests_stats_prc["fulfilled_requests_of_total_prc"] = round(requests_stats["fulfilled_requests"][-1] * 100.0 / requests_stats["total_requests"][-1], 2)
    requests_stats_prc["total_main_requests_prc"] = round(requests_stats["total_main_requests"][-1] * 100.0 / requests_stats["total_requests"][-1], 2)
    requests_stats_prc["total_subrequests_prc"] = round(requests_stats["total_subrequests"][-1] * 100.0 / requests_stats["total_requests"][-1], 2)
    requests_stats_prc["fulfilled_main_requests_prc"] = round(requests_stats["fulfilled_main_requests"][-1] * 100.0 / requests_stats["fulfilled_requests"][-1], 2)
    requests_stats_prc["fulfilled_subrequests_prc"] = round(requests_stats["fulfilled_subrequests"][-1] * 100.0 / requests_stats["fulfilled_requests"][-1], 2)


    return requests_stats, requests_stats_prc

def get_donations_stats(run_end):

    donations_data = defaultdict(lambda: 0)

    for donation_id, donation in df['donation_responses'][run_end].items():
        if donation.state == DONATION_SUBMITTED:
            donations_data["submitted"] += 1
        
        if donation.state == DONATION_SELECTED:
            donations_data["submitted"] += 1
            donations_data["selected"] += 1

        if donation.state == DONATION_FINALIZED:
            donations_data["submitted"] += 1
            donations_data["selected"] += 1
            donations_data["finalized"] += 1

    donations_data_prc = {
        "submitted": 100.0,
        "selected": round(donations_data["selected"] * 100.0 / donations_data["submitted"], 2),
        "finalized": round(donations_data["finalized"] * 100.0 / donations_data["submitted"], 2)
    }

    return donations_data, donations_data_prc

# source: chatGPT
def dict_to_csv(f, d):
    with open(f, 'a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=d.keys())
        
        if file.tell() == 0:
            writer.writeheader()

        writer.writerow(d)

In [ ]:
stats_by_run = defaultdict(lambda: {})

# generate data for analysis
starts = [0]
ends = [TIMESTEPS * len(PSUBs)]

for i in range(len(starts)):
    requests_stats, requests_stats_prc = get_requests_stats(starts[i], ends[i])
    donations_stats, donations_stats_prc = get_donations_stats(ends[i])

    stats_by_run[i]["request_stats"] = requests_stats
    stats_by_run[i]["requests_stats_prc"] = requests_stats_prc
    stats_by_run[i]["donations_stats"] = donations_stats
    stats_by_run[i]["donations_stats_prc"] = donations_stats_prc

In [ ]:
# prepare data

all_data = defaultdict(lambda: {})

for run in stats_by_run.keys():
    all_data[run] = defaultdict(lambda: 0.0)
    all_data[run]["sim_id"] = run

    for stat_type, value in stats_by_run[run]["request_stats"].items():
        all_data[run][stat_type] = value[-1]


    for stat_type, value in stats_by_run[run]["requests_stats_prc"].items():
        all_data[run][stat_type] = value

    for stat_type, value in stats_by_run[run]["donations_stats"].items():
        all_data[run][stat_type + "_count"] = value
        all_data[run][stat_type + "_prc"] = stats_by_run[run]["donations_stats_prc"][stat_type]


    all_data[run]["latency"] = df['metrics'][ends[run]]['latency']
    all_data[run]["throughput"] = df['metrics'][ends[run]]['throughput']

    # sim params
    all_data[run]["need_threshold"] = system_params["need_threshold"][0]

    for key in initial_state_params.keys():
        all_data[run][key] = initial_state_params[key]

In [ ]:
# save simulation results
for run in stats_by_run.keys():
    dict_to_csv('path/to/file.csv', all_data[run])

# save selected analysis data
dict_to_csv('path/to/file.csv', all_data[0])
df.to_csv('path/to/file.csv', index=False)

## Analysis

In [ ]:
data_total_req = np.array(requests_stats["total_requests"])
data_main_req = np.array(requests_stats["total_main_requests"])
data_sub_req = np.array(requests_stats["total_subrequests"])
data_total_fulfilled_req = np.array(requests_stats["fulfilled_requests"])
data_fulfilled_main_req = np.array(requests_stats["fulfilled_main_requests"])
data_fulfilled_sub_req = np.array(requests_stats["fulfilled_subrequests"])

plt.figure(figsize=(18, 8))

times = np.array([i for i in range(TIMESTEPS + 1)])

plt.plot(times, data_total_req, label='total requests', linestyle='-', color="orange")
plt.plot(times, data_main_req, label='total main requests', linestyle='--', color="orange")
plt.plot(times, data_sub_req, label='total subrequests', linestyle=':', color="orange")
plt.plot(times, data_total_fulfilled_req, label='fulfilled requests', linestyle='-', color="green")
plt.plot(times, data_fulfilled_main_req, label='fulfilled main requests', linestyle='--', color="green")
plt.plot(times, data_fulfilled_sub_req, label='fulfilled subrequests', linestyle=':', color="green")


plt.xlabel('timestep')
plt.ylabel('requests')
plt.xticks(times)
plt.legend()
plt.title("Exploratory Analysis: submitted vs fulfilled requests (raw count)")

plt.show()

### 1. Resource decentralization at beginning of system (first 30 resources)

In [ ]:
resources_decentralization = defaultdict(lambda: {})

owners = defaultdict(lambda: 0)

for agent_id, agent in agents.items():
    for resource_id in agent.inventory.stock.keys():
        owners[resource_id] += 1

for resource_id in decentralization_indices.keys():
    
    resources_decentralization[resource_id]["owners"] = owners[resource_id]
    resources_decentralization[resource_id]["min_c"] = decentralization_indices[resource_id]["min_c"]
    resources_decentralization[resource_id]["max_c"] = decentralization_indices[resource_id]["max_c"]
    resources_decentralization[resource_id]["dec_index"] = decentralization_indices[resource_id]["decentralization_index"]
    resources_decentralization[resource_id]["con_index"] = decentralization_indices[resource_id]["concentration_index"]
    resources_decentralization[resource_id]["distr_index"] = decentralization_indices[resource_id]["distribution_index"]


In [ ]:
dis = []
xticks = []

fig, ax1 = plt.subplots(figsize=(16, 5))

counter = 1

for resource_id in resources_decentralization.keys():
    if counter > 30: # stop display after the first 30 resources
        break

    dis.append(resources_decentralization[resource_id]["dec_index"])
    
    p1 = [counter, resources_decentralization[resource_id]["max_c"] - resources_decentralization[resource_id]["min_c"]]
    p2 = [counter, 0]
    
    ax1.plot([p1[0], p2[0]], [p1[1], p2[1]], color="blue", marker='_', linestyle="-", alpha=0.5)

    ax1.bar(counter, resources_decentralization[resource_id]['distr_index'], color="gray", alpha=0.3, width=0.5)

    xticks.append(counter)
    counter += 1

ax1.scatter(xticks, dis, marker="o", color="green")
ax1.set_ylabel("concentration index (ownership range) \n resource owners (as % of total)\n decentralization index")
ax1.set_xticks(xticks)
ax1.set_xticklabels(["r_"+str(i - 1)+"" for i in xticks])
ax1.set_yticklabels([0.0, 0.2, 0.4, 0.6, 0.8, 1.0], color="green")

ax1.set_xlabel("resource id")

fig.tight_layout() 
plt.title("initial state: resource decentralization (30 resources)")
plt.show()

In [ ]:
resources_decentralization_final = defaultdict(lambda: {})

owners_final = defaultdict(lambda: 0)

final_step = TIMESTEPS * len(PSUBs)
for agent_id, agent in df['agents'][final_step].items():
    for resource_id in agent.inventory.stock.keys():
        owners_final[resource_id] += 1

for resource_id in df['metrics'][final_step]['decentralization_index'].keys():
    
    resources_decentralization_final[resource_id]["owners"] = owners_final[resource_id]
    resources_decentralization_final[resource_id]["min_c"] = df['metrics'][final_step]['decentralization_index'][resource_id]["min_c"]
    resources_decentralization_final[resource_id]["max_c"] = df['metrics'][final_step]['decentralization_index'][resource_id]["max_c"]
    resources_decentralization_final[resource_id]["dec_index"] = df['metrics'][final_step]['decentralization_index'][resource_id]["decentralization_index"]
    resources_decentralization_final[resource_id]["con_index"] = df['metrics'][final_step]['decentralization_index'][resource_id]["concentration_index"]
    resources_decentralization_final[resource_id]["distr_index"] = df['metrics'][final_step]['decentralization_index'][resource_id]["distribution_index"]


In [ ]:
dis = []
xticks = []

fig, ax1 = plt.subplots(figsize=(16, 5))

counter = 1
for resource_id in resources_decentralization_final.keys():
    if counter > 30: # stop display after the first 30 resources
        break

    dis.append(resources_decentralization_final[resource_id]["dec_index"])
    
    p1 = [counter, resources_decentralization_final[resource_id]["max_c"] - resources_decentralization_final[resource_id]["min_c"]]
    p2 = [counter, 0]
    ax1.plot([p1[0], p2[0]], [p1[1], p2[1]], color="blue", marker='_', linestyle="-", alpha=0.5)

    ax1.bar(counter, resources_decentralization_final[resource_id]['distr_index'], color="gray", alpha=0.3, width=0.5)

    xticks.append(counter)
    counter += 1

ax1.scatter(xticks, dis, marker="o", color="green")
ax1.set_ylabel("concentration index (ownership range) \n resource owners (as % of total)\n decentralization index")
ax1.set_xticks(xticks)
ax1.set_xticklabels(["r_"+str(i - 1)+"" for i in xticks])
ax1.set_yticklabels([0.0, 0.2, 0.4, 0.6, 0.8, 1.0], color="green")

ax1.set_xlabel("resource id")



fig.tight_layout() 
plt.title("final state: resource decentralization (30 resources)")
plt.show()

### 2. Resource decentralization evolution (first 10 resources)

In [ ]:
resources_decentralization_by_state = defaultdict(lambda: {})

for state_id in range(TIMESTEPS):
    
    # owners for a given state
    owners = defaultdict(lambda: 0)

    for agent_id, agent in df['agents'][state_id * 5].items():
        for resource_id in agent.inventory.stock.keys():
            owners[resource_id] += 1

    st = "state_" + str(state_id)

    resources_decentralization_by_state[st] = defaultdict(lambda: {})
    decentralization_indices = df['metrics'][state_id * 5]['decentralization_index']

    for resource_id in decentralization_indices.keys():
        
        resources_decentralization_by_state[st][resource_id]["owners"] = owners[resource_id]
        resources_decentralization_by_state[st][resource_id]["min_c"] = decentralization_indices[resource_id]["min_c"]
        resources_decentralization_by_state[st][resource_id]["max_c"] = decentralization_indices[resource_id]["max_c"]
        resources_decentralization_by_state[st][resource_id]["dec_index"] = decentralization_indices[resource_id]["decentralization_index"]
        resources_decentralization_by_state[st][resource_id]["con_index"] = decentralization_indices[resource_id]["concentration_index"]
        resources_decentralization_by_state[st][resource_id]["distr_index"] = decentralization_indices[resource_id]["distribution_index"]


#### Resource_0 evolution

In [ ]:
rid = "resource_0"

dis = []
xticks = []

plt.figure(figsize=(16, 5))

for state_id in range(TIMESTEPS):

    st = "state_" + str(state_id)

    dis.append(resources_decentralization_by_state[st][rid]["dec_index"])
    

    p1 = [state_id, resources_decentralization_by_state[st][rid]["max_c"] - resources_decentralization_by_state[st][rid]["min_c"]]
    p2 = [state_id, 0]
    plt.plot([p1[0], p2[0]], [p1[1], p2[1]], color="blue", marker='_', linestyle="-", alpha=0.5)
    plt.bar(state_id, resources_decentralization_by_state[st][rid]['distr_index'], color="gray", alpha=0.3, width=0.5)

    xticks.append(state_id)
 
plt.scatter(xticks, dis, marker="o", color="green")# linestyle="-"
plt.ylabel("resource ownership range \n resource owners (as % of total)\n decentralization index")
plt.xticks(xticks)
plt.xlabel("state id")
plt.show()


In [ ]:
ctr = 0
dis = defaultdict(lambda: [])
xticks = []
rids = list(resources_decentralization_by_state["state_0"].keys())[:10]

plt.figure(figsize=(16, 5))


for state_id in range(TIMESTEPS):

    st = "state_" + str(state_id)

    for rid in rids:
        dis[rid].append(resources_decentralization_by_state[st][rid]["dec_index"])
    
    xticks.append(state_id)

ctr, ctr2 = 0, 1

colors = {}
linestyles = {}

for rid in dis.keys():
    c = (2 * ctr, 0.3 * ctr, 0.1 + 2 * ctr)

    colors[rid] = c

    l = "-"
    if ctr2 % 2 == 0:
        l = ":"
    if ctr2 % 3 == 0:
        l = "--"
    
    linestyles[rid] = l
    ctr += 0.05
    ctr2 += 1

for rid in dis.keys():

    plt.plot(xticks, dis[rid], linestyle=linestyles[rid], color=colors[rid])


plt.ylabel("decentralization index")
plt.xticks(xticks)
plt.xlabel("state id")
plt.legend(rids, loc=1)
plt.title("Exploratory Analysis: decentralization index evolution (10 resources)")
plt.show()


### 3. Decentralization-conscious agents' donations contributions

In [ ]:
state_id = TIMESTEPS * len(PSUBs)

agents_distribution = defaultdict(lambda: 0)

for agent_id, agent in df['agents'][state_id].items():
    agents_distribution[agent.atype] += 1

In [ ]:
donations_by_agent_type = defaultdict(lambda: 0)
finalized_donations_by_agent_type = defaultdict(lambda: 0)
selected_donations_by_agent_type = defaultdict(lambda: 0)

for donation_id, donation in df['donation_responses'][state_id].items():
    donations_by_agent_type[df['agents'][state_id][donation.donor].atype] += 1

    if donation.state == DONATION_FINALIZED:
        finalized_donations_by_agent_type[df['agents'][state_id][donation.donor].atype] += 1

donor_receipts_by_agent_type = defaultdict(lambda: 0)

for receipt_id, receipt in df['donation_receipts'][state_id].items():
    if receipt.rtype == RECEIPT_DONOR:
        donor_receipts_by_agent_type[df['agents'][state_id][receipt.agent_id].atype] += 1

In [ ]:
requests_stats = {
    "fulfilled_requests": 0,
    "fulfilled_main_requests": 0,
    "fulfilled_subrequests": 0,
    "total_requests": len(df['requests'][state_id]),
    "total_main_requests": 0,
    "total_subrequests": 0
}

for request_id, request in df['requests'][state_id].items():
    if request.rtype == "atomic" or request.rtype == "complex":
            requests_stats["total_main_requests"] += 1 

            if request.state == REQUEST_FULFILLED:
                requests_stats["fulfilled_main_requests"] += 1
    else:
        requests_stats["total_subrequests"] += 1

        if request.state == REQUEST_FULFILLED:
                requests_stats["fulfilled_subrequests"] += 1

requests_stats["fulfilled_requests"] = requests_stats["fulfilled_main_requests"] + requests_stats["fulfilled_subrequests"]

In [ ]:
# donor distribution
donation_distribution_data_by_agent = defaultdict(lambda: {})

# agent distribution
total_agents = sum(agents_distribution.values())
total_donations = sum(donations_by_agent_type.values())
total_finalized_donations = sum(finalized_donations_by_agent_type.values())
total_donor_receipts = sum(donor_receipts_by_agent_type.values())


for atype in agents_distribution.keys():
    donation_distribution_data_by_agent[atype]["prc_of_agents"] = round(agents_distribution[atype] * 100.0 / total_agents, 2)
    donation_distribution_data_by_agent[atype]["prc_of_donation_responses"] = round(donations_by_agent_type[atype] * 100.0 / total_donations, 2)
    donation_distribution_data_by_agent[atype]["prc_of_finalized_donations"] = round(finalized_donations_by_agent_type[atype] * 100.0 / total_finalized_donations, 2)
    donation_distribution_data_by_agent[atype]["prc_of_donor_receipts"] = round(donor_receipts_by_agent_type[atype] * 100.0 / total_donor_receipts, 2)


### Donations by economy

In [ ]:
donations_database = {}

for donation_id, donation in df['donation_responses'][state_id].items():
    if donation.state == DONATION_FINALIZED:
        donations_database[donation_id] = {
            "from": donation.donor,
            "to": df['requests'][state_id][donation.request_id].requestor,
            "econ_out": donation.economy_id_from,
            "econ_in": donation.economy_id_to,
            "resource_id": df['requests'][state_id][donation.request_id].resource_id,
            "quantity": donation.quantity
        }

In [ ]:
donations_database = {}

for donation_id, donation in df['donation_responses'][state_id].items():
    if donation.state == DONATION_FINALIZED:
        donations_database[donation_id] = {
            "from": donation.donor,
            "to": df['requests'][state_id][donation.request_id].requestor,
            "econ_out": donation.economy_id_from,
            "econ_in": donation.economy_id_to,
            "resource_id": df['requests'][state_id][donation.request_id].resource_id,
            "quantity": donation.quantity
        }

### Graph viz

In [ ]:
G = nx.MultiDiGraph()
H = nx.MultiDiGraph()

H.add_nodes_from(G.nodes)

for economy in economies:
    total_balance = 0
    G.add_node(economy, balance=total_balance)

for donation_id, transaction in donations_database.items():
    
    source = transaction['econ_out']
    destination = transaction['econ_in']
    amount = transaction['quantity']
    
    if source != destination:
        # update nodes' balances
        G.nodes[source]['balance'] -= amount
        G.nodes[destination]['balance'] += amount
        
        # add tx as edge
        G.add_edge(source, destination, amount=amount)
        


In [ ]:
# help with only drawing specific edges to help longitudinal analysis
# https://stackoverflow.com/questions/74406226/draw-specific-edges-in-graph-in-networkx
# help wtih animations
# https://stackoverflow.com/questions/43445103/inline-animations-in-jupyter

fig, ax = plt.subplots(figsize=(20, 15))

nodes_balances = nx.get_node_attributes(G, 'balance')
node_labels = {
    node: node.split("_")[1]
    for node, value
    in nodes_balances.items()
}

# node sizes
sizes = [700 for i in range(len(economies))]

pos = nx.spring_layout(G, seed=5)

def animate(t):
    plt.cla()

    nx.draw_networkx_nodes(G,
        pos, 
        node_size=sizes, 
        alpha=1,
        node_color="darkgreen")
        
    nx.draw_networkx_edges(H, pos, 
            alpha=0.05,
            width=3, 
            edge_color="gray")
    
    H.add_edge('econ_3', 'econ_4')

    donations_database = {}

    for donation_id, donation in df['donation_responses'][t * 5].items():
        if donation.state == DONATION_FINALIZED:
            donations_database[donation_id] = {
                "econ_out": donation.economy_id_from,
                "econ_in": donation.economy_id_to,
            }

    for donation_id, transaction in donations_database.items():
    
        source = transaction['econ_out']
        destination = transaction['econ_in']
        H.add_edge(source, destination, amount=amount)

    nx.draw_networkx_labels(G, pos, labels=node_labels, font_color="white", font_size=20)

anim = matplotlib.animation.FuncAnimation(fig, animate, frames=TIMESTEPS)

f = r"../animation.gif" 
writergif = matplotlib.animation.PillowWriter(fps=1) 
anim.save(f, writer=writergif)

In [ ]:
plt.figure().clear()
plt.close()
plt.cla()
plt.clf()